In [4]:
import cv2
import numpy as np
import os
import glob

#Paths
test_images_path = r"C:/Users/Fourth/data/SEGMENTATION DATA SET/ISIC2018_Task1-2_Validation_Input"
ground_truth_path = r"C:/Users/Fourth/data/SEGMENTATION DATA SET/ISIC2018_Task1_Validation_GroundTruth"

#Function to compute Dice score 
def dice_score(pred, target):
    pred = pred.astype(bool)
    target = target.astype(bool)
    intersection = np.logical_and(pred, target).sum()
    return (2. * intersection) / (pred.sum() + target.sum() + 1e-8)

#Contour Segmentation
def contour_segmentation(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, np.mean(gray), 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return np.zeros_like(gray)
    cnt = sorted(contours, key=cv2.contourArea)[-1]
    mask = np.zeros(gray.shape, dtype="uint8")
    cv2.drawContours(mask, [cnt], -1, 255, -1)
    return mask

#Watershed Segmentation
def watershed_segmentation(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
    sure_bg = cv2.dilate(opening, kernel, iterations=3)
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)
    ret, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown == 255] = 0
    markers = cv2.watershed(img.copy(), markers)
    binary_mask = np.uint8(markers > 1) * 255
    return binary_mask

#Load test images
test_images = glob.glob(os.path.join(test_images_path, "*.jpg"))
if not test_images:
    print("No test images found!")
    exit()

contour_scores = []
watershed_scores = []

print("Comparing Dice Scores for Contour vs Watershed:\n")

for img_path in test_images:
    filename = os.path.basename(img_path).replace(".jpg", "")
    gt_mask_path = os.path.join(ground_truth_path, filename + "_segmentation.png")

    if not os.path.exists(gt_mask_path):
        print(f"Ground truth not found for {filename}, skipping...")
        continue

    #Load and preprocess image
    img = cv2.imread(img_path)
    img = cv2.resize(img, (300, 300))  # resize to 300x300
    gt_mask = cv2.imread(gt_mask_path, cv2.IMREAD_GRAYSCALE)
    gt_mask = cv2.resize(gt_mask, (300, 300))
    _, gt_mask = cv2.threshold(gt_mask, 127, 255, cv2.THRESH_BINARY)

    #Get predicted masks
    contour_mask = contour_segmentation(img)
    watershed_mask = cv2.resize(watershed_segmentation(cv2.resize(img, (700, 700))), (300, 300))

    #Compute Dice scores
    d_contour = dice_score(contour_mask, gt_mask)
    d_watershed = dice_score(watershed_mask, gt_mask)

    contour_scores.append(d_contour)
    watershed_scores.append(d_watershed)

    print(f"{filename}: Contour Dice = {d_contour:.4f}, Watershed Dice = {d_watershed:.4f}")

#Print average Dice scores
print("\nAverage Dice Scores:")
print(f"Contour: {np.mean(contour_scores):.4f}")
print(f"Watershed: {np.mean(watershed_scores):.4f}")


Comparing Dice Scores for Contour vs Watershed:

ISIC_0012255: Contour Dice = 0.0822, Watershed Dice = 0.5110
ISIC_0012346: Contour Dice = 0.1737, Watershed Dice = 0.0870
ISIC_0012576: Contour Dice = 0.0000, Watershed Dice = 0.0000
ISIC_0012585: Contour Dice = 0.9619, Watershed Dice = 0.6054
ISIC_0012623: Contour Dice = 0.4551, Watershed Dice = 0.0319
ISIC_0012627: Contour Dice = 0.3499, Watershed Dice = 0.3427
ISIC_0012633: Contour Dice = 0.7487, Watershed Dice = 0.1228
ISIC_0012643: Contour Dice = 0.9723, Watershed Dice = 0.4728
ISIC_0015294: Contour Dice = 0.9512, Watershed Dice = 0.7642
ISIC_0015351: Contour Dice = 0.8825, Watershed Dice = 0.9138
ISIC_0015370: Contour Dice = 0.9652, Watershed Dice = 0.6209
ISIC_0015462: Contour Dice = 0.8691, Watershed Dice = 0.2258
ISIC_0015480: Contour Dice = 0.7706, Watershed Dice = 0.3528
ISIC_0015492: Contour Dice = 0.5070, Watershed Dice = 0.5073
ISIC_0015518: Contour Dice = 0.9550, Watershed Dice = 0.8200
ISIC_0015552: Contour Dice = 0.7935,